# Genie Enhancement v3 - Debug Notebook

## Three-Space Architecture with Batch Apply

**Safe enhancement workflow:**
1. Clone production → dev-working + dev-best
2. Score benchmarks on dev-working (baseline)
3. Apply ALL fixes at once to dev-working
4. Wait for indexing
5. Score again and compare
6. Promote dev-working → production (user decision)
7. Cleanup dev spaces

**Key Safety Features:**
- Production is NEVER modified directly
- All changes tested on dev-working first
- User controls final promotion

## 1️⃣ Setup

In [ ]:
# IMPORTANT: Clear cached modules to ensure latest code is loaded
import sys

modules_to_remove = [m for m in sys.modules if m.startswith('lib')]
for m in modules_to_remove:
    del sys.modules[m]

print(f"Cleared {len(modules_to_remove)} cached lib modules")

In [ ]:
# Project path setup
import sys
import os
from pathlib import Path

# Find project root
current_path = Path(os.getcwd())
if current_path.name == 'genie_enhancer':
    project_root = current_path
else:
    project_root = current_path
    while project_root.name != 'genie_enhancer' and project_root != project_root.parent:
        project_root = project_root.parent

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Configure logging
import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(name)s | %(levelname)s | %(message)s',
    datefmt='%H:%M:%S'
)

# Verbose logging for lib modules
for module in ['lib.genie_client', 'lib.scorer', 'lib.llm', 'lib.enhancer', 
               'lib.applier', 'lib.space_api', 'lib.space_cloner']:
    logging.getLogger(module).setLevel(logging.DEBUG)

print(f"Project root: {project_root}")
print(f"Logging: DEBUG mode enabled")

In [ ]:
# Imports
import json
import time
from datetime import datetime

from lib.genie_client import GenieConversationalClient
from lib.space_cloner import SpaceCloner
from lib.scorer import BenchmarkScorer
from lib.benchmark_parser import BenchmarkLoader
from lib.llm import DatabricksLLMClient
from lib.sql import SQLExecutor
from lib.category_enhancer import CategoryEnhancer  # NEW: Category-based analysis
from lib.applier import BatchApplier

print("✅ All imports successful")

## 2️⃣ Configuration

In [ ]:
# === UPDATE THESE VALUES ===
DATABRICKS_HOST = "your-workspace.cloud.databricks.com"
DATABRICKS_TOKEN = "YOUR_TOKEN_HERE"
GENIE_SPACE_ID = "your-space-id"  # Production space to enhance
WAREHOUSE_ID = "your-warehouse-id"  # For metric views
LLM_ENDPOINT = "databricks-claude-sonnet-4"

# Enhancement settings
TARGET_SCORE = 0.90
INDEXING_WAIT = 60  # seconds to wait after applying all fixes

print(f"Host: {DATABRICKS_HOST}")
print(f"Production Space: {GENIE_SPACE_ID}")
print(f"Warehouse: {WAREHOUSE_ID}")
print(f"LLM: {LLM_ENDPOINT}")
print(f"Target: {TARGET_SCORE:.0%}")

## 3️⃣ Initialize Clients

In [ ]:
# Space Cloner (for three-space architecture)
print("Initializing Space Cloner...")
space_cloner = SpaceCloner(
    host=DATABRICKS_HOST,
    token=DATABRICKS_TOKEN
)
print("✅ Space Cloner initialized")

In [ ]:
# LLM Client (with rate limit protection)
print("Initializing LLM Client...")
llm_client = DatabricksLLMClient(
    host=DATABRICKS_HOST,
    token=DATABRICKS_TOKEN,
    endpoint_name=LLM_ENDPOINT,
    request_delay=10.0,          # 10s delay between requests
    rate_limit_base_delay=90.0   # 90s base on rate limit
)

if llm_client.test_connection():
    print("✅ LLM Client connected")
    print("   - Request delay: 10s")
    print("   - Rate limit backoff: 90s base")
else:
    print("❌ LLM connection failed")

In [ ]:
# SQL Executor (for metric views)
print("Initializing SQL Executor...")
sql_executor = SQLExecutor(
    host=DATABRICKS_HOST,
    token=DATABRICKS_TOKEN,
    warehouse_id=WAREHOUSE_ID
)
print("✅ SQL Executor initialized")

## 4️⃣ Load Benchmarks

In [ ]:
# Load benchmarks
benchmark_file = project_root / "benchmarks" / "fixed_benchmark_final.json"
print(f"Loading from: {benchmark_file}")

loader = BenchmarkLoader(str(benchmark_file))
all_benchmarks = loader.load()
print(f"✅ Loaded {len(all_benchmarks)} benchmarks")

# Preview
for i, b in enumerate(all_benchmarks[:3]):
    print(f"  {i+1}. {b['question'][:60]}...")

In [ ]:
# Optional: Use subset for faster testing
USE_SUBSET = True  # Set to False for full run

if USE_SUBSET:
    benchmarks = all_benchmarks[:5]  # First 5 only
    print(f"⚠️ TEST MODE: Using {len(benchmarks)} benchmarks")
else:
    benchmarks = all_benchmarks
    print(f"FULL MODE: Using {len(benchmarks)} benchmarks")

---
## 5️⃣ Setup Three-Space Architecture

This creates:
- **Production** - Original space (never modified)
- **Dev-Working** - Where changes are tested
- **Dev-Best** - Backup of production (for rollback)

In [ ]:
print("="*60)
print("SETTING UP THREE-SPACE ARCHITECTURE")
print("="*60)
print()
print(f"Production Space: {GENIE_SPACE_ID}")
print("Creating dev-working and dev-best clones...")
print()

setup_result = space_cloner.setup_three_spaces(
    production_space_id=GENIE_SPACE_ID
)

if setup_result['success']:
    print()
    print("="*60)
    print("THREE-SPACE ARCHITECTURE READY")
    print("="*60)
    print(f"Production:   {setup_result['production_id']}")
    print(f"Dev-Working:  {setup_result['dev_working_id']}")
    print(f"Dev-Best:     {setup_result['dev_best_id']}")
    
    # Store for later
    PRODUCTION_ID = setup_result['production_id']
    DEV_WORKING_ID = setup_result['dev_working_id']
    DEV_BEST_ID = setup_result['dev_best_id']
    INITIAL_CONFIG = setup_result['initial_config']
else:
    print(f"❌ Setup failed: {setup_result['error']}")
    raise RuntimeError("Three-space setup failed")

## 6️⃣ Initialize Scorer and Planner

In [ ]:
# Genie Client pointing to DEV-WORKING space
print("Initializing Genie Client for dev-working space...")
genie_client = GenieConversationalClient(
    host=DATABRICKS_HOST,
    token=DATABRICKS_TOKEN,
    space_id=DEV_WORKING_ID,  # Point to dev-working, NOT production
    verbose=True
)
print(f"✅ Genie Client initialized (space: {DEV_WORKING_ID[:16]}...)")

In [ ]:
# Benchmark Scorer
print("Initializing Scorer...")
scorer = BenchmarkScorer(
    genie_client=genie_client,
    llm_client=llm_client,
    sql_executor=sql_executor,
    config={
        "question_timeout": 120,
        "question_delay": 3.0,
        "error_delay": 5.0,
        "parallel_workers": 0,  # Sequential for debugging
    }
)
print("✅ Scorer initialized (sequential mode)")

In [ ]:
# Category-Based Enhancement Planner (NEW!)
# Uses 9 LLM calls total instead of N failures × 3 categories
print("Initializing Category Enhancer...")
prompts_dir = project_root / "prompts"
planner = CategoryEnhancer(llm_client, prompts_dir)
print("✅ Category Enhancer initialized")
print("   - 9 fix categories (constant LLM calls)")
print("   - Categories: instruction, metadata, sample_queries, sql_snippets, join_specs")

In [ ]:
# Batch Applier (applies to dev-working via space_cloner)
print("Initializing Batch Applier...")
applier = BatchApplier(
    space_api=space_cloner,  # Use space_cloner as the API
    sql_executor=sql_executor,
    config={
        "catalog": "sandbox",
        "schema": "genie_enhancement"
    }
)
print("✅ Batch Applier initialized")

---
## 7️⃣ Initial Scoring (Baseline)

In [ ]:
print("="*60)
print("INITIAL SCORING (baseline on dev-working)")
print("="*60)
print()

start_time = datetime.now()
initial_results = scorer.score(benchmarks)
duration = (datetime.now() - start_time).total_seconds()

initial_score = initial_results['score']

print()
print("="*60)
print("BASELINE SCORING COMPLETE")
print("="*60)
print(f"Baseline Score: {initial_score:.1%}")
print(f"Passed: {initial_results['passed']}/{initial_results['total']}")
print(f"Failed: {initial_results['failed']}")
print(f"Duration: {duration:.1f}s")

# Check if already at target
if initial_score >= TARGET_SCORE:
    print()
    print("🎉 Already at target score! No enhancement needed.")

In [ ]:
# Show failed benchmarks
failed_results = [r for r in initial_results['results'] if not r['passed']]

print(f"\n❌ Failed Benchmarks ({len(failed_results)}):\n")
for i, r in enumerate(failed_results, 1):
    print(f"{i}. {r['question'][:60]}...")
    print(f"   Category: {r.get('failure_category', 'unknown')}")
    if r.get('failure_reason'):
        print(f"   Reason: {r['failure_reason'][:80]}")
    print()

---
## 8️⃣ Generate Enhancement Plan

In [ ]:
print("="*60)
print("GENERATING ENHANCEMENT PLAN (Category-Based)")
print("="*60)
print()
print(f"Analyzing {len(failed_results)} failures...")
print("9 Categories: instruction_fix, metadata_add/delete, sample_queries_add/delete,")
print("              sql_snippets_add/delete, join_specs_add/delete")
print()

plan_start = datetime.now()
grouped_fixes = planner.generate_plan(
    failed_benchmarks=failed_results,
    space_config=INITIAL_CONFIG,
    parallel_workers=3  # Can run categories in parallel
)
plan_duration = (datetime.now() - plan_start).total_seconds()

# New 9 categories
FIX_CATEGORIES = [
    'instruction_fix',
    'join_specs_delete', 'join_specs_add',
    'sql_snippets_delete', 'sql_snippets_add',
    'metadata_delete', 'metadata_add',
    'sample_queries_delete', 'sample_queries_add',
]

total_fixes = sum(len(grouped_fixes.get(cat, [])) for cat in FIX_CATEGORIES)

print()
print("="*60)
print("PLAN GENERATION COMPLETE")
print("="*60)
print(f"Total fixes: {total_fixes}")
print(f"Duration: {plan_duration:.1f}s")
print(f"LLM calls: 9 (constant, regardless of failure count)")

for category in FIX_CATEGORIES:
    count = len(grouped_fixes.get(category, []))
    if count > 0:
        print(f"  - {category}: {count}")

In [ ]:
# Preview fixes by category
print("\nFix Preview:")
print("-"*40)

for category in FIX_CATEGORIES:
    fixes = grouped_fixes.get(category, [])
    if fixes:
        print(f"\n{category.upper()} ({len(fixes)} fixes):")
        for i, fix in enumerate(fixes[:5], 1):  # Show first 5
            fix_type = fix.get('type', 'unknown')
            # Metadata fixes
            if fix_type == 'add_synonym':
                print(f"  {i}. {fix_type}: {fix.get('table')}.{fix.get('column')} → '{fix.get('synonym')}'")
            elif fix_type == 'delete_synonym':
                print(f"  {i}. {fix_type}: {fix.get('table')}.{fix.get('column')} ✗ '{fix.get('synonym')}'")
            elif fix_type in ('add_column_description', 'add_table_description'):
                print(f"  {i}. {fix_type}: {fix.get('table')}")
            # Sample query fixes
            elif fix_type == 'add_example_query':
                print(f"  {i}. {fix_type}: {fix.get('pattern_name', 'N/A')}")
            elif fix_type == 'delete_example_query':
                print(f"  {i}. {fix_type}: {fix.get('pattern_name', fix.get('id', 'N/A'))}")
            # Instruction fixes
            elif fix_type == 'update_text_instruction':
                print(f"  {i}. {fix_type}")
            # SQL snippet fixes
            elif fix_type in ('add_filter', 'add_expression', 'add_measure'):
                print(f"  {i}. {fix_type}: {fix.get('display_name', fix.get('alias', 'N/A'))}")
            elif fix_type in ('delete_filter', 'delete_expression', 'delete_measure'):
                print(f"  {i}. {fix_type}: {fix.get('display_name', fix.get('id', 'N/A'))}")
            # Join spec fixes
            elif fix_type == 'add_join_spec':
                print(f"  {i}. {fix_type}: {fix.get('left_table')} ↔ {fix.get('right_table')}")
            elif fix_type == 'delete_join_spec':
                print(f"  {i}. {fix_type}: {fix.get('left_table')} ↔ {fix.get('right_table')}")
            else:
                print(f"  {i}. {fix_type}")
        if len(fixes) > 5:
            print(f"  ... and {len(fixes) - 5} more")

---
## 9️⃣ Apply ALL Fixes at Once (Batch)

This applies ALL fixes to dev-working in one batch:
1. Apply all fixes at once
2. Wait for Genie indexing
3. Score and compare with baseline

In [ ]:
print("="*60)
print("APPLYING ALL FIXES (BATCH MODE)")
print("="*60)
print()
print(f"Total fixes to apply: {total_fixes}")
print(f"Target space: dev-working ({DEV_WORKING_ID[:16]}...)")
print()

apply_start = datetime.now()
apply_result = applier.apply_all(
    space_id=DEV_WORKING_ID,  # Apply to dev-working, NOT production
    grouped_fixes=grouped_fixes,
    dry_run=False  # Actually apply
)
apply_duration = (datetime.now() - apply_start).total_seconds()

print()
print("="*60)
print("BATCH APPLY COMPLETE")
print("="*60)
print(f"Applied: {len(apply_result['applied'])}")
print(f"Failed: {len(apply_result['failed'])}")
print(f"Duration: {apply_duration:.1f}s")

if apply_result['failed']:
    print("\n❌ Failed fixes:")
    for fix in apply_result['failed'][:5]:
        print(f"  - {fix.get('type')}: {fix.get('error', 'N/A')[:50]}")

In [ ]:
# Wait for Genie indexing
print(f"\nWaiting {INDEXING_WAIT}s for Genie to index changes...")
for i in range(INDEXING_WAIT, 0, -10):
    print(f"  {i}s remaining...")
    time.sleep(10)
print("✅ Indexing wait complete")

---
## 🔟 Validation Scoring (After Fixes)

In [ ]:
print("="*60)
print("VALIDATION SCORING (after fixes)")
print("="*60)
print()

val_start = datetime.now()
final_results = scorer.score(benchmarks)
val_duration = (datetime.now() - val_start).total_seconds()

final_score = final_results['score']
improvement = final_score - initial_score

print()
print("="*60)
print("VALIDATION COMPLETE")
print("="*60)
print()
print(f"Baseline Score: {initial_score:.1%} ({initial_results['passed']}/{initial_results['total']} passed)")
print(f"Final Score:    {final_score:.1%} ({final_results['passed']}/{final_results['total']} passed)")
print(f"Improvement:    {improvement:+.1%}")
print(f"Target:         {TARGET_SCORE:.1%}")
print()
print(f"Duration: {val_duration:.1f}s")
print()

if final_score >= TARGET_SCORE:
    print("🎉 TARGET REACHED!")
elif improvement > 0:
    print(f"📈 IMPROVED by {improvement:+.1%} (gap to target: {TARGET_SCORE - final_score:.1%})")
elif improvement == 0:
    print("➖ NO CHANGE - fixes may not have addressed the failures")
else:
    print(f"📉 REGRESSED by {improvement:.1%} - consider discarding changes")

In [ ]:
# Show what changed
print("\nDetailed comparison:")
print("-"*60)

for i, (before, after) in enumerate(zip(initial_results['results'], final_results['results']), 1):
    before_pass = before['passed']
    after_pass = after['passed']
    
    if before_pass != after_pass:
        if after_pass:
            status = "❌→✅ FIXED"
        else:
            status = "✅→❌ BROKEN"
        print(f"{i}. {status}: {before['question'][:50]}...")

---
## 1️⃣1️⃣ Promotion Decision

**Your options:**
1. **Promote** - Copy dev-working config to production
2. **Discard** - Delete dev spaces, keep production unchanged

In [ ]:
# Summary
print("="*60)
print("ENHANCEMENT SUMMARY")
print("="*60)
print()
print(f"Production Space: {PRODUCTION_ID} (unchanged)")
print(f"Dev-Working:      {DEV_WORKING_ID} (has fixes)")
print(f"Dev-Best:         {DEV_BEST_ID} (backup)")
print()
print(f"Baseline Score: {initial_score:.1%}")
print(f"Final Score:    {final_score:.1%}")
print(f"Improvement:    {improvement:+.1%}")
print(f"Fixes Applied:  {len(apply_result['applied'])}")
print()
print("Your decision:")
print("  • Set CONFIRM_PROMOTE = True to apply changes to production")
print("  • Set CONFIRM_CLEANUP = True to discard and keep production unchanged")

In [ ]:
# OPTION 1: Promote to Production
# This copies dev-working config to production

CONFIRM_PROMOTE = False  # Set to True to enable

if CONFIRM_PROMOTE:
    print("Promoting dev-working to production...")
    
    # Copy dev-working config to production
    promote_result = space_cloner.copy_config(
        source_space_id=DEV_WORKING_ID,
        target_space_id=PRODUCTION_ID
    )
    
    if promote_result['success']:
        print("✅ Production updated with enhanced configuration!")
        print(f"   New score: {final_score:.1%}")
        
        # Cleanup dev spaces
        print("\nCleaning up dev spaces...")
        cleanup_result = space_cloner.cleanup_dev_spaces()
        if cleanup_result['success']:
            print("✅ Dev spaces deleted")
        else:
            print(f"⚠️ Cleanup warning: {cleanup_result['error']}")
    else:
        print(f"❌ Promotion failed: {promote_result['error']}")
else:
    print("Set CONFIRM_PROMOTE = True to apply dev-working changes to production")

In [ ]:
# OPTION 2: Cleanup without promoting
# This discards all changes and keeps production unchanged

CONFIRM_CLEANUP = False  # Set to True to enable

if CONFIRM_CLEANUP:
    print("Cleaning up dev spaces (no changes to production)...")
    cleanup_result = space_cloner.cleanup_dev_spaces()
    
    if cleanup_result['success']:
        print("✅ Dev spaces deleted")
        print("Production space unchanged.")
    else:
        print(f"⚠️ Cleanup warning: {cleanup_result['error']}")
else:
    print("Set CONFIRM_CLEANUP = True to delete dev spaces and discard changes")

---
## Debug Utilities

In [ ]:
# Export results to JSON
output = {
    "production_id": PRODUCTION_ID,
    "dev_working_id": DEV_WORKING_ID,
    "dev_best_id": DEV_BEST_ID,
    "baseline_score": initial_score,
    "final_score": final_score,
    "improvement": improvement,
    "fixes_applied": len(apply_result['applied']),
    "fixes_failed": len(apply_result['failed']),
}

with open('enhancement_result.json', 'w') as f:
    json.dump(output, f, indent=2)
print("✅ Results saved to enhancement_result.json")

In [ ]:
# Export fixes to JSON for review
with open('debug_fixes.json', 'w') as f:
    json.dump(grouped_fixes, f, indent=2, default=str)
print("✅ Fixes saved to debug_fixes.json")

In [ ]:
# Test Genie on dev-working
test_question = "What tables are available?"
print(f"Testing Genie (dev-working): {test_question}")

response = genie_client.ask(test_question, timeout=60)
print(f"Status: {response['status']}")
if response.get('sql'):
    print(f"SQL: {response['sql'][:100]}...")